### Build the Neural Network
- 신경망은 데이터에 대한 연산을 수행하는 layers(계층)/modules(모듈) 로 구성되어 있다.<br>
`torch.nn` namespace 는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.<br>
PyTorch 의 모든 modules 는 `nn.Module` 의 subclass(하위 클래스) 이다.<br>
신경망은 다른 modules(layers) 로 구성된 module 이다. 이러한 nested structure(중첩된 구조) 는<br>
복잡한 architecture 를 쉽게 구축하고 관리할 수 있다.

In [18]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [19]:
# 학습을 위한 장치 얻기
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


#### 클래스 정의하기
- 신경망 모델을 `nn.Module` 의 subclass 로 정의하고, `__init__` 에서 신경망 계층들을 초기화한다.<br>
`nn.Module` 을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산들을 구현한다.

In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [24]:
X = torch.rand(1, 28, 28)
logits = model(X)
pred_probab = nn.Softmax(1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


#### 모델 계층(Layer)
- FashionMNIST 모델의 계층들을 각각 살펴보기위해 28 x 28 크기의 이미지 3개로 구성된 minibatch 를 가져와,<br>
신경망을 통과할 때 어떤 일이 발생하는지 알아보기

In [26]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [27]:
# nn.Flatten: 28 x 28 의 2D 영상을 784 픽셀 값을 갖는 1D 배열로 반환한다.
# 이 때 dim=0 의 minibatch 차원은 유지된다.
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [30]:
# nn.Linear
# linear 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을
# 적용하는 모듈이다.
layer1 = nn.Linear(28*28, 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [31]:
# nn.ReLU
# 
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.5355, -0.4473,  0.3768,  0.4140,  0.3452, -0.2701, -0.2226, -0.1164,
          0.6100, -0.0298,  0.0867,  0.2006, -0.1352, -0.2160, -0.5956,  0.1883,
          0.3782, -0.2448, -0.1244, -0.2186],
        [ 0.3681, -0.2529,  0.2594,  0.2858,  0.0838, -0.2304, -0.3866, -0.0099,
          0.3116,  0.2426, -0.0185, -0.0186,  0.1046, -0.2370, -0.5441,  0.4314,
          0.4622,  0.2673, -0.4008, -0.3181],
        [ 0.4651, -0.3663,  0.0704,  0.6769,  0.2379,  0.0259, -0.3038,  0.0255,
          0.5264,  0.1843, -0.0126, -0.0781,  0.1702,  0.0289, -0.8257,  0.1674,
          0.1141, -0.2245, -0.1461, -0.4462]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.5355, 0.0000, 0.3768, 0.4140, 0.3452, 0.0000, 0.0000, 0.0000, 0.6100,
         0.0000, 0.0867, 0.2006, 0.0000, 0.0000, 0.0000, 0.1883, 0.3782, 0.0000,
         0.0000, 0.0000],
        [0.3681, 0.0000, 0.2594, 0.2858, 0.0838, 0.0000, 0.0000, 0.0000, 0.3116,
         0.2426, 0.0000, 0.0000, 0.1046, 0.0000, 0.00

In [33]:
# nn.Sequential
# nn.Sequential 은 순서를 갖는 모듈의 container 이다.
# 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달된다.
# sequential container 를 사용하여 아래와 같이 신경망을 빠르게 만들 수 있다.
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.2362,  0.1134,  0.0634,  0.2016,  0.2382,  0.2145,  0.0105, -0.5061,
          0.1440,  0.1566],
        [ 0.1635,  0.1120,  0.0651,  0.2072,  0.0589,  0.0845,  0.0042, -0.3615,
          0.0991,  0.0791],
        [ 0.2728,  0.1067,  0.0020,  0.1893,  0.2406,  0.1815,  0.0375, -0.4241,
          0.2675,  0.1433]], grad_fn=<AddmmBackward0>)


In [34]:
# nn.Softmax
# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 [-inf, inf] 범위의 raw value 인 logits 를 반환한다.
# logits 는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)된다.
# dim 매개변수(parameter)는 값의 합이 1이 되는 차원을 나타낸다.
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1139, 0.1007, 0.0958, 0.1100, 0.1141, 0.1114, 0.0909, 0.0542, 0.1038,
         0.1052],
        [0.1108, 0.1052, 0.1004, 0.1157, 0.0998, 0.1024, 0.0945, 0.0655, 0.1039,
         0.1018],
        [0.1167, 0.0988, 0.0890, 0.1073, 0.1130, 0.1065, 0.0922, 0.0581, 0.1160,
         0.1025]], grad_fn=<SoftmaxBackward0>)


In [36]:
# 모델 매개변수
# 신경망 내부의 많은 계층들은 매개변수화(parameterize) 된다.
# 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어진다.
# nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며,
# 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 된다.
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0056,  0.0233,  0.0035,  ..., -0.0132,  0.0337,  0.0278],
        [-0.0215,  0.0298,  0.0311,  ...,  0.0329,  0.0125, -0.0091]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0208, -0.0329], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0163,  0.0277,  0.0209,  ...,  0.0352,  0.0106,  0.0259],
        [-0.0373,  0.0349,  0.0427,  ..., -0.0007, -0.0150, -0.0022]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Siz